In [1]:
import torch
import numpy as np

In [2]:
from pathlib import Path
import os

train_dir = Path("../dataset_subsets/re10k_subset/train")
chunk_id = 0
chunk = torch.load(train_dir / f'{chunk_id:06d}.torch')

/tmp/ipykernel_1065906/485655885.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  chunk = torch.load(train_dir / f'{chunk_id:06d}.torch')


In [3]:
from PIL import Image
from torchvision import transforms as tf
from jaxtyping import Float, UInt8
from torch import Tensor
from io import BytesIO
from typing import NamedTuple
from einops import rearrange
import math

def focal2fov(focal, pixels):
    # I think pixel size is assumed to be 1.0
    return 2*math.atan(pixels/(2*focal))

to_tensor = tf.ToTensor()

def convert_images(
    images: list[UInt8[Tensor, "..."]],
) -> Float[Tensor, "batch 3 height width"]:
    torch_images = []
    for image in images:
        image = Image.open(BytesIO(image.numpy().tobytes()))
        torch_images.append(to_tensor(image))
    return torch.stack(torch_images)

class CameraInfo(NamedTuple):
    uid: int
    R: np.array
    T: np.array
    FovY: np.array
    FovX: np.array
    image: np.array
    image_path: str
    image_name: str
    width: int
    height: int
    

def retrieve_cam_infos(poses: Float[Tensor, "batch 18"], images: Float[Tensor, "batch 3 height width"], scene_id: str) -> tuple[
    Float[Tensor, "batch 4 4"],  # extrinsics
    Float[Tensor, "batch 3 3"],  # intrinsics
]:
    cam_infos = []

    height, width = images.shape[2:]

    for idx, (pose, image) in enumerate(zip(poses, images)):
        # Convert the intrinsics to a 3x3 normalized K matrix.
        intrinsics = torch.eye(3, dtype=torch.float32)
        fx, fy, cx, cy = pose[:4]
        intrinsics[0, 0] = width * fx
        intrinsics[1, 1] = height * fy
        intrinsics[0, 2] = width * cx
        intrinsics[1, 2] = height * cy

        FovX = focal2fov(fx * width, width)
        FovY = focal2fov(fy * height, height)

        # TODO: I believe the w2c matrix is enough to acquire R and T.
        # Check this assumption!
        # Convert the extrinsics to a 4x4 OpenCV-style W2C matrix.
        w2c = torch.eye(4, dtype=torch.float32)
        w2c[:3] = rearrange(pose[6:], "(h w) -> h w", h=3, w=4)
        w2c = w2c.numpy()

        R = w2c[:3, :3]
        T = w2c[:3, 3]

        # Conver PyTorch image to PIL Image. This is required in `cameraList_from_camInfos` during
        # `Scene` class instantiation.
        image = Image.fromarray((image.permute(1, 2, 0) * 255).numpy().astype(np.uint8))

        # cam_info = CameraInfo(
        #     uid=idx, R=R, T=T, FovY=FovY, FovX=FovX, image=image,
        #     image_path=scene_id, image_name=f"{idx}", width=width, height=height
        # )
        cam_info = {
            "uid": idx,
            "intrinsics": intrinsics,
            "R": R,
            "T": T,
            "FovY": FovY,
            "FovX": FovX,
            "image": image,
            "image_path": scene_id,
            "image_name": f"{idx}",
            "width": width,
            "height": height
        }
        cam_infos.append(cam_info)

    return cam_infos

In [4]:
def rotmat2qvec(R: np.array) -> np.array:
    Rxx, Ryx, Rzx, Rxy, Ryy, Rzy, Rxz, Ryz, Rzz = R.flat
    K = np.array([
        [Rxx - Ryy - Rzz, 0, 0, 0],
        [Ryx + Rxy, Ryy - Rxx - Rzz, 0, 0],
        [Rzx + Rxz, Rzy + Ryz, Rzz - Rxx - Ryy, 0],
        [Ryz - Rzy, Rzx - Rxz, Rxy - Ryx, Rxx + Ryy + Rzz]]) / 3.0
    eigvals, eigvecs = np.linalg.eigh(K)
    qvec = eigvecs[[3, 0, 1, 2], np.argmax(eigvals)]
    if qvec[0] < 0:
        qvec *= -1
    return qvec

def qvec2rotmat(qvec):
    return np.array([
        [1 - 2 * qvec[2]**2 - 2 * qvec[3]**2,
         2 * qvec[1] * qvec[2] - 2 * qvec[0] * qvec[3],
         2 * qvec[3] * qvec[1] + 2 * qvec[0] * qvec[2]],
        [2 * qvec[1] * qvec[2] + 2 * qvec[0] * qvec[3],
         1 - 2 * qvec[1]**2 - 2 * qvec[3]**2,
         2 * qvec[2] * qvec[3] - 2 * qvec[0] * qvec[1]],
        [2 * qvec[3] * qvec[1] - 2 * qvec[0] * qvec[2],
         2 * qvec[2] * qvec[3] + 2 * qvec[0] * qvec[1],
         1 - 2 * qvec[1]**2 - 2 * qvec[2]**2]])

In [5]:
# chosen_sample_key = "d358cb839f4c7497"
# chosen_sample_key = "63c4a6e3f38760e2"
chosen_sample_key = "67bd3eefd07e6042"

save_dir = f"scene_{chosen_sample_key}"

if os.path.exists(save_dir):
    os.system(f"rm -rf {save_dir}")
os.makedirs(save_dir, exist_ok=True)

all_cam_infos = []

samples = {}

for sample in chunk:

    if sample['key'] != chosen_sample_key:
        continue

    print(sample['key'], sample['url'])

    tensor_images = []
    np_images = []
    print(f"Number of images: {len(sample['images'])}")

    vis = True

    images = convert_images(sample['images'])
    cam_infos = retrieve_cam_infos(sample['cameras'], images, sample['key'])

    # print(cam_infos)
    print(images.shape)

    if sample['key'] not in samples:
        samples[sample['key']] = {
            'images': images,
            'cameras': cam_infos
        }
    else:
        print(f"Sample {sample['key']} already exists!")

    os.makedirs(f"{save_dir}/images", exist_ok=True)
    for idx, cam_info in enumerate(cam_infos):
        pil_image = cam_info['image']
        pil_image.save(f"{save_dir}/images/{idx:04d}.png")
        
        # if vis:
        #     print(f"Images have size {np_image.shape}")
        #     plt.imshow(np_image)
        #     plt.axis('off')
        #     plt.show()

        #     vis = False

    # tensor_images = torch.stack(tensor_images)
    # np_images = np.stack(np_images)

    # all_cam_infos.extend(cam_infos)

67bd3eefd07e6042 https://www.youtube.com/watch?v=ERh1pPZcRQE
Number of images: 174
torch.Size([174, 3, 360, 640])


In [6]:
import os

print(f"Save dir: {save_dir}")

os.makedirs(f"{save_dir}/sparse", exist_ok=True)

if os.path.exists(f"{save_dir}/sparse/cameras.txt"):
    os.remove(f"{save_dir}/sparse/cameras.txt")
    os.system(f"touch {save_dir}/sparse/cameras.txt")
if os.path.exists(f"{save_dir}/sparse/images.txt"):
    os.remove(f"{save_dir}/sparse/images.txt")
    os.system(f"touch {save_dir}/sparse/images.txt")

os.system(f"touch {save_dir}/sparse/points3D.txt")

# Create a template which takes for the lines in cameras.txt
# Line format similar to: 
# 1 SIMPLE_PINHOLE 3072 2304 2559.81 1536 1152

intr_template = lambda idx, width, height, focal_length, cx, cy: \
    f"{idx} SIMPLE_PINHOLE {width} {height} {focal_length} {cx} {cy}"

# Create a template for the lines in images.txt
# Line format similar to:
# IMAGE_ID, QW, QX, QY, QZ, TX, TY, TZ, CAMERA_ID, NAME

extr_template = lambda idx, qw, qx, qy, qz, tx, ty, tz, cam_id, name: \
    f"{idx} {qw} {qx} {qy} {qz} {tx} {ty} {tz} {cam_id} {name}"

# # Create 100 cameras with random parameters
# num_cameras = 100
# cameras = [template(idx, 3072, 2304, 2559.81, 1536, 1152) for idx in range(1, num_cameras + 1)]

for key, sample in samples.items():
    if key != chosen_sample_key:
        continue

    print(f"Sample {key} has {len(sample['cameras'])} cameras")
    for cam_info, image in zip(sample['cameras'], sample['images']):
        
        ### Write camera intrinsics to cameras.txt
        focal_length = (
            cam_info['intrinsics'][0, 0].item() + \
            cam_info['intrinsics'][1, 1].item()
        ) / 2.0

        cam_line = intr_template(
            cam_info['uid'] + 1, 
            cam_info['width'], 
            cam_info['height'], 
            f"{focal_length:.2f}", 
            int(cam_info['intrinsics'][0, 2].item()), 
            int(cam_info['intrinsics'][1, 2].item())
        )

        # Write cameras to cameras.txt
        with open(f"{save_dir}/sparse/cameras.txt", 'a') as file:
            file.write(cam_line + '\n')

        ### Write camera extrinsics to images.txt
        # print(cam_info['R'], cam_info['image_name'])
        quat = rotmat2qvec(cam_info['R'])
        trans = cam_info['T']

        extr_line = extr_template(
            cam_info['uid'] + 1, 
            quat[0], quat[1], quat[2], quat[3], 
            trans[0], trans[1], trans[2], 
            cam_info['uid'] + 1, 
            f"{cam_info['uid']:04d}.png"
        )

        # Write extrinsics to images.txt
        with open(f"{save_dir}/sparse/images.txt", 'a') as file:
            file.write(extr_line + '\n')
            file.write('\n')


Save dir: scene_67bd3eefd07e6042
Sample 67bd3eefd07e6042 has 174 cameras


In [7]:
import os

print(f"Save dir: {save_dir}")

try:
    os.remove(f"{save_dir}/database.db")
    os.remove(f"{save_dir}/database.db-shm")
    os.remove(f"{save_dir}/database.db-wal")
except:
    print("Database files do not exist!")

os.system(f"""
colmap feature_extractor \
    --ImageReader.camera_model SIMPLE_PINHOLE \
    --database_path {save_dir}/database.db \
    --image_path {save_dir}/images
"""
)

Save dir: scene_67bd3eefd07e6042
Database files do not exist!


I0916 23:41:55.003556 1066307 misc.cc:198] 
Feature extraction
I0916 23:41:55.003993 1066316 sift.cc:718] Creating SIFT GPU feature extractor
I0916 23:41:55.134923 1066317 feature_extraction.cc:257] Processed file [1/174]
I0916 23:41:55.134936 1066317 feature_extraction.cc:260]   Name:            0000.png
I0916 23:41:55.134938 1066317 feature_extraction.cc:269]   Dimensions:      640 x 360
I0916 23:41:55.134939 1066317 feature_extraction.cc:272]   Camera:          #1 - SIMPLE_PINHOLE
I0916 23:41:55.134941 1066317 feature_extraction.cc:275]   Focal Length:    768.00px
I0916 23:41:55.134946 1066317 feature_extraction.cc:279]   Features:        1914
I0916 23:41:55.143543 1066317 feature_extraction.cc:257] Processed file [2/174]
I0916 23:41:55.143553 1066317 feature_extraction.cc:260]   Name:            0001.png
I0916 23:41:55.143555 1066317 feature_extraction.cc:269]   Dimensions:      640 x 360
I0916 23:41:55.143556 1066317 feature_extraction.cc:272]   Camera:          #2 - SIMPLE_PINHOL

0

In [8]:
from database import COLMAPDatabase
import pycolmap

print(f"Save dir: {save_dir}")

database_path = f"{save_dir}/database.db"
pycolmap_database = pycolmap.Database(database_path)

Save dir: scene_67bd3eefd07e6042


In [9]:
for key, sample in samples.items():
    if key != chosen_sample_key:
        continue

    print(f"Sample {key} has {len(sample['cameras'])} cameras")
    for cam_info, image in zip(sample['cameras'], sample['images']):
        
        ### Write camera intrinsics to cameras.txt
        focal_length = (
            cam_info['intrinsics'][0, 0].item() + \
            cam_info['intrinsics'][1, 1].item()
        ) / 2.0

        cam = pycolmap.Camera.create(
            camera_id=cam_info['uid'] + 1,
            model=0,        #SIMPLE_PINHOLE
            focal_length=focal_length,
            width=cam_info['width'],
            height=cam_info['height']
        )
        pycolmap_database.update_camera(cam)

pycolmap_database.close()

Sample 67bd3eefd07e6042 has 174 cameras


In [10]:
print(f"Save dir: {save_dir}")

os.system(f"colmap exhaustive_matcher --database_path {save_dir}/database.db")

Save dir: scene_67bd3eefd07e6042


I0916 23:41:56.539113 1066333 misc.cc:198] 
Feature matching
I0916 23:41:56.539670 1066334 sift.cc:1423] Creating SIFT GPU feature matcher
I0916 23:41:56.633747 1066333 pairing.cc:168] Generating exhaustive image pairs...
I0916 23:41:56.633757 1066333 pairing.cc:201] Matching block [1/4, 1/4]
I0916 23:41:56.940937 1066333 feature_matching.cc:46] in 0.307s
I0916 23:41:56.951431 1066333 pairing.cc:201] Matching block [1/4, 2/4]
I0916 23:41:57.218063 1066333 feature_matching.cc:46] in 0.267s
I0916 23:41:57.219128 1066333 pairing.cc:201] Matching block [1/4, 3/4]
I0916 23:41:57.565261 1066333 feature_matching.cc:46] in 0.346s
I0916 23:41:57.566020 1066333 pairing.cc:201] Matching block [1/4, 4/4]
I0916 23:41:57.648564 1066333 feature_matching.cc:46] in 0.083s
I0916 23:41:57.648819 1066333 pairing.cc:201] Matching block [2/4, 1/4]
I0916 23:41:57.862553 1066333 feature_matching.cc:46] in 0.214s
I0916 23:41:57.868839 1066333 pairing.cc:201] Matching block [2/4, 2/4]
I0916 23:41:58.081847 1066

0

In [11]:
print(f"Save dir: {save_dir}")

os.system(f"""
colmap point_triangulator \
    --database_path {save_dir}/database.db \
    --image_path {save_dir}/images \
    --input_path {save_dir}/sparse/ \
    --output_path {save_dir}/sparse/"""
)

I0916 23:42:00.487505 1066352 misc.cc:198] 
Loading model
I0916 23:42:00.489223 1066352 incremental_mapper.cc:232] Loading database
I0916 23:42:00.489698 1066352 database_cache.cc:65] Loading cameras...
I0916 23:42:00.489758 1066352 database_cache.cc:75]  174 in 0.000s
I0916 23:42:00.489761 1066352 database_cache.cc:83] Loading matches...


Save dir: scene_67bd3eefd07e6042


I0916 23:42:00.516462 1066352 database_cache.cc:89]  11747 in 0.027s
I0916 23:42:00.516479 1066352 database_cache.cc:105] Loading images...
I0916 23:42:00.522307 1066352 database_cache.cc:155]  174 in 0.006s (connected 174)
I0916 23:42:00.522328 1066352 database_cache.cc:166] Building correspondence graph...
I0916 23:42:00.905089 1066352 database_cache.cc:195]  in 0.383s (ignored 0)
I0916 23:42:00.906205 1066352 timer.cc:91] Elapsed time: 0.007 [minutes]
I0916 23:42:00.910432 1066352 incremental_mapper.cc:554] Iterative triangulation
I0916 23:42:00.910439 1066352 incremental_mapper.cc:560] Triangulating image #1 (0)
I0916 23:42:00.910444 1066352 incremental_mapper.cc:562] => Image sees 0 / 1706 points
I0916 23:42:01.001768 1066352 incremental_mapper.cc:560] Triangulating image #2 (1)
I0916 23:42:01.001780 1066352 incremental_mapper.cc:562] => Image sees 443 / 1732 points
I0916 23:42:01.055122 1066352 incremental_mapper.cc:560] Triangulating image #3 (2)
I0916 23:42:01.055130 1066352 in

0

In [12]:
import struct
import numpy as np
import collections

CameraModel = collections.namedtuple(
    "CameraModel", ["model_id", "model_name", "num_params"])
Camera = collections.namedtuple(
    "Camera", ["id", "model", "width", "height", "params"])
BaseImage = collections.namedtuple(
    "Image", ["id", "qvec", "tvec", "camera_id", "name", "xys", "point3D_ids"])
Point3D = collections.namedtuple(
    "Point3D", ["id", "xyz", "rgb", "error", "image_ids", "point2D_idxs"])
CAMERA_MODELS = {
    CameraModel(model_id=0, model_name="SIMPLE_PINHOLE", num_params=3),
    CameraModel(model_id=1, model_name="PINHOLE", num_params=4),
    CameraModel(model_id=2, model_name="SIMPLE_RADIAL", num_params=4),
    CameraModel(model_id=3, model_name="RADIAL", num_params=5),
    CameraModel(model_id=4, model_name="OPENCV", num_params=8),
    CameraModel(model_id=5, model_name="OPENCV_FISHEYE", num_params=8),
    CameraModel(model_id=6, model_name="FULL_OPENCV", num_params=12),
    CameraModel(model_id=7, model_name="FOV", num_params=5),
    CameraModel(model_id=8, model_name="SIMPLE_RADIAL_FISHEYE", num_params=4),
    CameraModel(model_id=9, model_name="RADIAL_FISHEYE", num_params=5),
    CameraModel(model_id=10, model_name="THIN_PRISM_FISHEYE", num_params=12)
}
CAMERA_MODEL_IDS = dict([(camera_model.model_id, camera_model)
                         for camera_model in CAMERA_MODELS])
CAMERA_MODEL_NAMES = dict([(camera_model.model_name, camera_model)
                           for camera_model in CAMERA_MODELS])

class Image(BaseImage):
    def qvec2rotmat(self):
        return qvec2rotmat(self.qvec)

def read_next_bytes(fid, num_bytes, format_char_sequence, endian_character="<"):
    """Read and unpack the next bytes from a binary file.
    :param fid:
    :param num_bytes: Sum of combination of {2, 4, 8}, e.g. 2, 6, 16, 30, etc.
    :param format_char_sequence: List of {c, e, f, d, h, H, i, I, l, L, q, Q}.
    :param endian_character: Any of {@, =, <, >, !}
    :return: Tuple of read and unpacked values.
    """
    data = fid.read(num_bytes)
    return struct.unpack(endian_character + format_char_sequence, data)

def read_points3D_binary(path_to_model_file):
    """
    see: src/base/reconstruction.cc
        void Reconstruction::ReadPoints3DBinary(const std::string& path)
        void Reconstruction::WritePoints3DBinary(const std::string& path)
    """


    with open(path_to_model_file, "rb") as fid:
        num_points = read_next_bytes(fid, 8, "Q")[0]

        xyzs = np.empty((num_points, 3))
        rgbs = np.empty((num_points, 3))
        errors = np.empty((num_points, 1))

        for p_id in range(num_points):
            binary_point_line_properties = read_next_bytes(
                fid, num_bytes=43, format_char_sequence="QdddBBBd")
            xyz = np.array(binary_point_line_properties[1:4])
            rgb = np.array(binary_point_line_properties[4:7])
            error = np.array(binary_point_line_properties[7])
            track_length = read_next_bytes(
                fid, num_bytes=8, format_char_sequence="Q")[0]
            track_elems = read_next_bytes(
                fid, num_bytes=8*track_length,
                format_char_sequence="ii"*track_length)
            xyzs[p_id] = xyz
            rgbs[p_id] = rgb
            errors[p_id] = error
    return xyzs, rgbs, errors

def read_extrinsics_binary(path_to_model_file):
    """
    see: src/base/reconstruction.cc
        void Reconstruction::ReadImagesBinary(const std::string& path)
        void Reconstruction::WriteImagesBinary(const std::string& path)
    """
    images = {}
    with open(path_to_model_file, "rb") as fid:
        num_reg_images = read_next_bytes(fid, 8, "Q")[0]
        for _ in range(num_reg_images):
            binary_image_properties = read_next_bytes(
                fid, num_bytes=64, format_char_sequence="idddddddi")
            image_id = binary_image_properties[0]
            qvec = np.array(binary_image_properties[1:5])
            tvec = np.array(binary_image_properties[5:8])
            camera_id = binary_image_properties[8]
            image_name = ""
            current_char = read_next_bytes(fid, 1, "c")[0]
            while current_char != b"\x00":   # look for the ASCII 0 entry
                image_name += current_char.decode("utf-8")
                current_char = read_next_bytes(fid, 1, "c")[0]
            num_points2D = read_next_bytes(fid, num_bytes=8,
                                           format_char_sequence="Q")[0]
            x_y_id_s = read_next_bytes(fid, num_bytes=24*num_points2D,
                                       format_char_sequence="ddq"*num_points2D)
            xys = np.column_stack([tuple(map(float, x_y_id_s[0::3])),
                                   tuple(map(float, x_y_id_s[1::3]))])
            point3D_ids = np.array(tuple(map(int, x_y_id_s[2::3])))
            images[image_id] = Image(
                id=image_id, qvec=qvec, tvec=tvec,
                camera_id=camera_id, name=image_name,
                xys=xys, point3D_ids=point3D_ids)
    return images

In [13]:
import open3d as o3d

print(f"Save dir: {save_dir}")

xyzs, rgbs, errors = read_points3D_binary(f"{save_dir}/sparse/points3D.bin")
print(xyzs.shape)
print(rgbs.shape)
print(errors.shape)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyzs)
pcd.colors = o3d.utility.Vector3dVector(rgbs / 255.0)

# Save the point cloud to a ply file
o3d.io.write_point_cloud(f"{save_dir}/points3D.ply", pcd)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Save dir: scene_67bd3eefd07e6042
(5920, 3)
(5920, 3)
(5920, 1)


True

In [14]:
images = read_extrinsics_binary(f"{save_dir}/sparse/images.bin")
print(len(images))
print(images.keys())
print(images[1])

174
dict_keys([127, 126, 125, 124, 123, 122, 121, 120, 119, 118, 117, 116, 115, 114, 113, 112, 111, 110, 109, 108, 107, 106, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174])
Image(id=1, qvec=array([ 9.97213615e-01,  7.22604083e-05, -7.34735845e-02, -1.29086594e-02]), tvec=array([-0.03923461, -0.0259432 , -0.12889408]), camera_id=1, name='0000.png', xys=array([[554.06640625,   2.27172017],
    

In [15]:
os.makedirs(f"{save_dir}/sparse/0", exist_ok=True)
os.system(f"mv {save_dir}/sparse/*.txt {save_dir}/sparse/0/")
os.system(f"mv {save_dir}/sparse/*.bin {save_dir}/sparse/0/")

0